# Prepare Temperature datasets before indexing to Cambodia cube

This notebook can be used to clip multiple raster in a folder and clip to a shapefile. The output file is given the same name with the prefix of "clipped_"

In [1]:
import pandas as pd
import dask
from distributed import Client, LocalCluster

In [10]:
#Import libraries
import os
import glob
import gdal

In [45]:
#Set locations
nc_location= '/g/data/u46/users/ext547/ewater/input_data/test/'
clipped_nc_location = '/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/'
cambodia_shp='/g/data/u46/users/ext547/ewater/input_data/Cambodia_boundary/KHM_adm0.shp'

infile='/g/data/u46/users/ext547/ewater/input_data/test/clipped_test_1979-01-01T00:00:00.nc'
outfile='/g/data/u46/users/ext547/ewater/input_data/test/degreesC_test_1979-01-01T00:00:00.nc'

In [57]:
#create a glob listing all netcdf
nc_files = glob.glob(nc_location + '*.nc')

In [12]:
# #loop through all netcdf in glob and clip with shapefile
# for nc in nc_files:
#     name=nc[-20:] #save name of file
#     print(nc)
#     print(name)
#     clipped_name = str(clipped_nc_location+'clipped_'+name) #give new name with "clipped"
#     print(clipped_name)
#     gdal.Warp(srcDSOrSrcDSTab=nc,destNameOrDestDS=clipped_name, cutlineDSName= cambodia_shp, 
#               cropToCutline=True, format = 'netcdf') #clip to shapefile and save

In [15]:
# !gdal_calc.py -A /g/data/u46/users/ext547/ewater/input_data/test/clipped_test_1979-01-01T00:00:00.nc NoDataValue=-9999 dtype='Float32' --calc=A-273.15 --outfile=/g/data/u46/users/ext547/ewater/input_data/test/test_blah.nc

In [14]:
import netCDF4
dset = netCDF4.Dataset(nc_location+'clipped_test_1979-01-01T00:00:00.nc')
print(dset.variables['Band1'])
# dset['Band1'][:][dset['Band1'][:] == 0] = -9999
# print(dset['Band1'])
# dset.close()

<class 'netCDF4._netCDF4.Variable'>
float32 Band1(lat, lon)
    long_name: GDAL Band Number 1
    _FillValue: 9.96921e+36
    units: K
unlimited dimensions: 
current shape = (10, 11)
filling off



In [2]:
from cdo import *
cdo=Cdo()
cdo.CDO = '/apps/cdo/1.7.1/bin/cdo'     #need to update the location of cdo or else the wrapper can't find cdo.exe
cdo

In [48]:
help(cdo.setvals)

Help on method get in module cdo:

get(*args, **kwargs) method of cdo.Cdo instance
    NAME
        setvals, setrtoc, setrtoc2 - Replace variable values
    
    SYNOPSIS
        setvals,oldval,newval[,...]  ifile ofile
        setrtoc,rmin,rmax,c  ifile ofile
        setrtoc2,rmin,rmax,c,c2  ifile ofile
    
    DESCRIPTION
        This module replaces old variable values with new values, depending on the operator.
    
    OPERATORS
        setvals   Set list of old values to new values
                  Supply a list of n pairs of old and new values.
        setrtoc   Set range to constant
                           / c      if i(t,x) GE rmin AND i(t,x) LE rmax
                  o(t,x) = 
                           \ i(t,x) if i(t,x) LT rmin AND i(t,x) GT rmax
        setrtoc2  Set range to constant others to constant2
                           / c      if i(t,x) GE rmin AND i(t,x) LE rmax
                  o(t,x) = 
                           \ c2     if i(t,x) LT rmin AND i(t,x) 

In [56]:
blah = cdo.setvals(oldval='0',newval='-9999',ifile=infile,ofile=outfile)

Error in calling operator setvals with:
>>> /apps/cdo/1.7.1/bin/cdo -O setvals /local/u46/ext547/tmp/cdoPyzx6zq83p<<<
STDOUT:

STDERR:cdo setvals (Abort): Too few streams specified! Operator needs 1 input and 1 output streams.



CDOException: (returncode:1) cdo setvals (Abort): Too few streams specified! Operator needs 1 input and 1 output streams.


In [43]:
help(cdo.setvals)

Help on method get in module cdo:

get(*args, **kwargs) method of cdo.Cdo instance
    NAME
        setvals, setrtoc, setrtoc2 - Replace variable values
    
    SYNOPSIS
        setvals,oldval,newval[,...]  ifile ofile
        setrtoc,rmin,rmax,c  ifile ofile
        setrtoc2,rmin,rmax,c,c2  ifile ofile
    
    DESCRIPTION
        This module replaces old variable values with new values, depending on the operator.
    
    OPERATORS
        setvals   Set list of old values to new values
                  Supply a list of n pairs of old and new values.
        setrtoc   Set range to constant
                           / c      if i(t,x) GE rmin AND i(t,x) LE rmax
                  o(t,x) = 
                           \ i(t,x) if i(t,x) LT rmin AND i(t,x) GT rmax
        setrtoc2  Set range to constant others to constant2
                           / c      if i(t,x) GE rmin AND i(t,x) LE rmax
                  o(t,x) = 
                           \ c2     if i(t,x) LT rmin AND i(t,x) 

In [ ]:
cdo setvals,0,-9999 *.nc *.nc

In [39]:
for nc in nc_files:
     name=nc[-20:] #save name of file
     print(nc)
     print(name)
     clipped_name = str(clipped_nc_location+'degreesC_'+name) #give new name with "clipped"
     print(clipped_name)
     ! cdo setvals,0,-9999 nc.nc clipped_name.nc

/g/data/u46/users/ext547/ewater/input_data/test/test_1979-01-08T09:00:00.nc
79-01-08T09:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-08T09:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/clipped_test_1979-01-01T15:00:00.nc
79-01-01T15:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-01T15:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/clipped_test_1979-01-20T21:00:00.nc
79-01-20T21:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-20T21:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/test_1979-01-30T00:00:00.nc
79-01-30T00:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-30T00:00:00.nc

/g/data/u46/users/ext547/ewater/input_data/test/test_1979-01-10T15:00:00.nc
79-01-10T15:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-10T15:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/clipped_test_1979-01-24T03:00:00.nc
79-01-24T03:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-24T03:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/test_1979-01-14T03:00:00.nc
79-01-14T03:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-14T03:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/clipped_test_1979-01-24T15:00:00.nc
79-01-24T15:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-24T15:00:00.nc

/g/data/u46/users/ext547/ewater/input_data/test/test_1979-01-29T15:00:00.nc
79-01-29T15:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-29T15:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/test_1979-01-27T21:00:00.nc
79-01-27T21:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-27T21:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/clipped_test_1979-01-14T00:00:00.nc
79-01-14T00:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-14T00:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/test_1979-01-02T03:00:00.nc
79-01-02T03:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-02T03:00:00.nc

cdo se

/g/data/u46/users/ext547/ewater/input_data/test/clipped_test_1979-01-28T03:00:00.nc
79-01-28T03:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-28T03:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/test_1979-01-02T21:00:00.nc
79-01-02T21:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-02T21:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/clipped_test_1979-01-20T06:00:00.nc
79-01-20T06:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-20T06:00:00.nc

cdo setvals: Open failed on >nc.nc<
No such file or directory
/g/data/u46/users/ext547/ewater/input_data/test/test_1979-01-25T15:00:00.nc
79-01-25T15:00:00.nc
/g/data/u46/users/ext547/ewater/input_data/Temperature/Clipped_Cambodia/degreesC_79-01-25T15:00:00.nc

UnboundLocalError: local variable 'child' referenced before assignment

In [24]:
cdo.readArray(dset,dset.variables['Band1'])

Could not import data from file '<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.5
    GDAL: GDAL 2.2.4, released 2018/03/19
    history: Wed May 23 14:33:54 2018: GDAL Create( /g/data/u46/users/ext547/ewater/input_data/test/clipped_test_1979-01-01T00:00:00.nc, ... )
    dimensions(sizes): lon(11), lat(10)
    variables(dimensions): float32 Band1(lat,lon), float64 lat(lat), float64 lon(lon)
    groups: 
'


FileNotFoundError: [Errno 2] No such file or directory: b"<class 'netCDF4._netCDF4.Dataset'>\nroot group (NETCDF3_CLASSIC data model, file format NETCDF3):\n    Conventions: CF-1.5\n    GDAL: GDAL 2.2.4, released 2018/03/19\n    history: Wed May 23 14:33:54 2018: GDAL Create( /g/data/u46/users/ext547/ewater/input_data/test/clipped_test_1979-01-01T00:00:00.nc, ... )\n    dimensions(sizes): lon(11), lat(10)\n    variables(dimensions): float32 \x1b[4mBand1\x1b[0m(lat,lon), float64 \x1b[4mlat\x1b[0m(lat), float64 \x1b[4mlon\x1b[0m(lon)\n    groups: \n"

In [17]:
help(cdo)

Help on Cdo in module cdo object:

class Cdo(builtins.object)
 |  Methods defined here:
 |  
 |  __dir__(self)
 |      __dir__() -> list
 |      default dir() implementation
 |  
 |  __getattr__(self, method_name)
 |  
 |  __init__(self, returnCdf=False, returnNoneOnError=False, forceOutput=True, cdfMod='netcdf4', env=environ({'MODULE_VERSION_STACK': '3.2.10', 'MANPATH': '/opt/pbs/default/share/man:/usr/local/share/man:/usr/share/man:/usr/X11R6/man:/opt/man', 'ORBIT_SOCKETDIR': '/local/u46/ext547/tmp/orbit-ext547', 'VNCDESKTOP': 'vdi-n3:2 (ext547)', 'SSH_AGENT_PID': '18291', 'HOSTNAME': 'vdi-n3', 'NF_MODULES_LOADED': 'YES', 'TERM': 'xterm-color', 'SHELL': '/bin/bash', 'CDO_ROOT': '/apps/cdo/1.9.2_cmor-3.2.8', 'XDG_SESSION_COOKIE': '0c9274f9cb1df2af6912555a0000009f-1526858817.135134-1199378231', 'PROJECT': 'u46', 'CDO_BASE': '/apps/cdo/1.9.2_cmor-3.2.8', 'TMPDIR': '/local/u46/ext547/tmp', 'GTK_RC_FILES': '/etc/gtk/gtkrc:/home/547/ext547/.gtkrc-1.2-gnome2', 'WINDOWID': '44697017', 'FPATH

In [16]:
# !gdalinfo /g/data/u46/users/ext547/ewater/input_data/test/test_1979-01-31T21:00:00.nc